In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from __future__ import division, print_function
from __future__ import absolute_import, unicode_literals
from IPython.display import display
import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

import bokeh.io
import bokeh.mpl
import bokeh.plotting

%config InlineBackend.figure_formats = {'eps', 'retina'}
bokeh.io.output_notebook()

rc = {'lines.linewidth': 1, 
      'axes.labelsize': 14, 
      'axes.titlesize': 14, 
      'axes.facecolor': 'FFFFFF',
       }

sns.set_context('notebook', rc=rc)
sns.set_style('white', rc=rc)
sns.set_palette("colorblind")

pd.set_option('precision', 4)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
def is_bad(x):
    if x in [-0.5, -1.0, 9.5]:
        return True
    else:
        return False

def load_transform(fp):
    df = pd.read_json(fp)
    df = df.sort_values(by=['trip', 't'])
    bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
    df['trip_interval'] = pd.cut(df['trip'], 
                                    bins=bins,
                                    precision=3,
                                    labels = trips_intervals)
    df['success'] = df['success'].astype(int)
    df['reward'] = df['reward'].astype(int)
    df['bad_moves'] = df['reward'].apply(lambda x: int(is_bad(x)))
    return df


def plot_path(df, deadline=False):
    df = df.sort_values(by=['trip', 't'])
    grid = sns.FacetGrid(df, col="trip", 
                         hue="trip",
                         col_wrap=5,
                         size=2.0,
                         aspect=1.5)
    grid.map(plt.axhline, y=0, ls=":", c=".5", label='No move')
    grid.map(plt.plot, "t", "reward", marker="o", ms=5, lw=.8)

    if deadline:
        grid.set(xticks=np.linspace(0, 50, 6, endpoint=True), xlim=(-1, 50), ylim=(-3, 13))
    else:
        grid.set(xticks=np.linspace(0, 150, 6, endpoint=True), xlim=(-1, 150), ylim=(-3, 13))
    grid.fig.tight_layout(w_pad=1)
    plt.savefig("./figures/informed_agent.eps", format='eps', dpi=1000)

def review_results(df):
    print('Successful trips:')
    display(pd.crosstab(df.trip_interval, df.success, margins=True).T)
    
    print("\nRewards:")
    display(pd.crosstab(df.trip_interval, df.reward, margins=True).T)

    print("\nBad moves:")
    display(pd.crosstab(df.trip_interval, df.bad_moves, margins=True).T)
    
def publish_results(df):
    print(tabulate(df.groupby(['trip_interval'])['success', 'bad_moves'].sum(),
         headers=['Trips', 'Success', 'Bad moves'],
         tablefmt='latex_booktabs'))
    
trips_intervals = ['01 - 10', '11 - 20', '21 - 30', '31 - 40', '41 - 50', '51 - 60', '61 - 70', '71 - 80', '81 - 90', '91 - 100']

In [9]:
# File needed for the notebook
fp = "./results/informed_agent.json"
df = load_transform(fp)

In [10]:
# Random Action No Deadline
review_results(df)

Successful trips:


trip_interval,01 - 10,11 - 20,21 - 30,31 - 40,41 - 50,51 - 60,61 - 70,71 - 80,81 - 90,91 - 100,All
success,,,,,,,,,,,
0,63,709,318,865,466,83,292,843,457,241,4337
1,6,5,6,4,6,6,9,1,6,8,57
All,69,714,324,869,472,89,301,844,463,249,4394



Rewards:


trip_interval,01 - 10,11 - 20,21 - 30,31 - 40,41 - 50,51 - 60,61 - 70,71 - 80,81 - 90,91 - 100,All
reward,,,,,,,,,,,
-1,0,2,0,0,1,0,0,0,1,0,4
0,33,43,75,77,58,50,115,82,48,76,657
2,30,664,244,789,407,33,178,761,410,165,3681
12,6,5,5,3,6,6,8,1,4,8,52
All,69,714,324,869,472,89,301,844,463,249,4394



Bad moves:


trip_interval,01 - 10,11 - 20,21 - 30,31 - 40,41 - 50,51 - 60,61 - 70,71 - 80,81 - 90,91 - 100,All
bad_moves,,,,,,,,,,,
0,69,712,324,869,471,89,301,844,462,249,4390
1,0,2,0,0,1,0,0,0,1,0,4
All,69,714,324,869,472,89,301,844,463,249,4394


In [11]:
publish_results(df)

\begin{tabular}{lrr}
\toprule
 Trips    &   Success &   Bad moves \\
\midrule
 01 - 10  &         6 &           0 \\
 11 - 20  &         5 &           2 \\
 21 - 30  &         6 &           0 \\
 31 - 40  &         4 &           0 \\
 41 - 50  &         6 &           1 \\
 51 - 60  &         6 &           0 \\
 61 - 70  &         9 &           0 \\
 71 - 80  &         1 &           0 \\
 81 - 90  &         6 &           1 \\
 91 - 100 &         8 &           0 \\
\bottomrule
\end{tabular}


In [12]:
first_20 = df[df['trip'] <= 20]
plot_path(first_20, deadline=False)

In [13]:
informed_agent_dealine = load_transform("./results/informed_agent_deadline.json")
publish_results(informed_agent_dealine)

\begin{tabular}{lrr}
\toprule
 Trips    &   Success &   Bad moves \\
\midrule
 01 - 10  &         5 &           0 \\
 11 - 20  &         4 &           0 \\
 21 - 30  &         5 &           0 \\
 31 - 40  &         5 &           1 \\
 41 - 50  &         2 &           0 \\
 51 - 60  &         7 &           0 \\
 61 - 70  &         3 &           0 \\
 71 - 80  &         4 &           1 \\
 81 - 90  &         6 &           1 \\
 91 - 100 &         4 &           0 \\
\bottomrule
\end{tabular}
